# caldav_to_papierkram

Ich hasse Zeiterfassung. Ich habe nie verstanden, warum man in einem eigenen Tool Kunden, Jobnummern, Stunden und Beschreibungen erfassen muss, wenn man das alles doch schon zumindest grob in seinem Kalender pflegt.
caldav_to_papierkram sucht in einem Kalender alle Termine eines Monats und versucht in Papierkram die entsprechenden Kunden und Jobs zu finden um dann Time Entries für Papierkram zu erzeugen und diese per API hochzuladen. 

Im Kalender werden die Zeiten erfasst und nach dem Format "Firma - Projekt - Tätigkeit - Beschreibung" beschriftet.
Also sowas wie: "Meier GmbH - Website - Programmierung - Layout überarbeitet und Templates produktiv gestellt"

**Nie mehr "Stunden buchen", nur noch "Kalender pflegen".**

## Vorbereitungen

Dieses Notebook braucht und will nicht viel. Man muss einen API Zugang bei Papierkram haben. Außerdem braucht es folgende Python Pakete:

- pip install caldav
- pip install python-dotenv

Die Credentials müssen in einem .env File im Verzeichnis des Notebooks liegen.
Außerdem braucht man einen Zugang zur Papierkram API. Siehe: https://api-doc.papierkram.de/api/v1/api-docs/index.html

In [213]:

from dotenv import load_dotenv
load_dotenv() 
import os

# Die Zugangsdaten für CalDav hängen vom Provider/Software ab.
caldav_url = os.environ["CALDAV_URL"]
username = os.environ["CALDAV_USER"]
password = os.environ["CALDAV_PASSWORD"]

# Die exzellente Papierkram Doku findet man hier: https://api-doc.papierkram.de/api/v1/api-docs/index.html
papierkram_endpoint = os.environ["PAPIERKRAM_ENDPOINT"]
papierkram_token = os.environ["PAPIERKRAM_TOKEN"]

USER_ID = 1     # Papierkram User ID für den gebucht werden soll (bei Freelance Einzelkämpfern in der Regel die 1. Im Zweifel weiter unten per getUsers() einmal nachschauen.)
year = 2024     # Jahr für das gebucht werden soll
month = 1       # Monat für den gebucht werden soll


# Mich nervt es, wenn ich im Kalender immer "Meier GmbH" schreiben muss. Daher kann man hier eine Liste mit Kurzformen der eigenen Kunden pflegen, dann verwende tman im Kalender die Kurzform und hier wird entsprechend ersetzt.
short_companies = {
    "MEIER": {
        "long":"Michael Meier GmbH & Co. KG",
        "id": 1
    },
    "Schmidt": {
        "long":"Schmidt, Schneider udn Söhne GmbH & Co. KG aA GbR",
        "id": 2
    }
}


# Einlesen CalDav Events

In [ ]:
import caldav
import calendar
from datetime import datetime
import json


# Erster Tag ab 00:00 Uhr
first_day = datetime(year, month, 1, 0, 0)

# Letzter Tag bis 23:59 Uhr
last_day_in_month = calendar.monthrange(year, month)[1]
last_day = datetime(year, month, last_day_in_month, 23, 59)


time_entries = []
sum_duration = 0 

with caldav.DAVClient( url=caldav_url, username=username, password=password ) as client:

    calendar = client.calendar(url=caldav_url)

    events = calendar.search(
        start=first_day,
        end=last_day,
        event=True,
        expand=True,
    )
    
    if len(events) > 1:
        
        for e in events:
                    
            # Aufbau das Timesheet Entries
            timesheet  = {
                "entry_date": None,
                "started_at_time": None,
                "ended_at_time": None,
                "duration": None,
                "company": None,
                "project": None,
                "task": None,
                "comments": "",
            }
                    
            # Datum und Uhrzeite eintragen
            timesheet["entry_date"] = datetime.strftime( e.icalendar_component["DTSTART"].dt, "%Y-%m-%d" )
            timesheet["started_at_time"] = datetime.strftime( e.icalendar_component["DTSTART"].dt, "%H:%M" )
            timesheet["ended_at_time"] = datetime.strftime( e.icalendar_component["DTEND"].dt, "%H:%M" )
            
            # In der Terminbeschreibung liegen im Format "Firma - Projekt - Tätigkeit - Beschreibung" alle Informationene in einem Feld.

            job = e.icalendar_component["SUMMARY"].split("-")

            for j in job:
                if job.index(j) == 0:
                    timesheet["company"] = j.strip()
                elif job.index(j) == 1:
                    timesheet["project"] = j.strip()
                elif job.index(j) == 2:
                    timesheet["task"] = j.strip()
                else:
                    timesheet["comments"] += j.strip()

            # Dauer eintragen
            duration = e.icalendar_component["DTEND"].dt - e.icalendar_component["DTSTART"].dt             
            timesheet["duration"] = duration
            
            # Dauer über alle Einträge im Monat summieren
            sum_duration += duration.total_seconds() / 3600

            # Fertig. Zur Liste aller Einträge hinzufügen
            time_entries.append( timesheet )
            
# Summe aller Einträge anzeigen
print( year, "-", month, " : ", sum_duration / 8 , " days" )


# Hilfsfunktionen um Endpoints in Papierkram abzufragen

In [216]:
import requests
import json

# Define the API endpoint and Token
api_url = papierkram_endpoint
bearer_token = papierkram_token

# Define headers with the Bearer token
headers = {
    'Authorization': f'Bearer {bearer_token}',
    'Accept': 'application/json'
}

endpoints = {
    "projects": "/projects",
    "tasks": "/tracker/tasks",
    "companies": "/contact/companies",
    "users": "/users"
}


def bookTimeEntry( time_entry ):
    try:
        response = requests.post(api_url + "/tracker/time_entries", headers=headers, json=time_entry )

        if response.status_code == 201:
            print( "Created. ", end="")
            heads = response.headers
            for key, value in heads.items():
                if key.lower() in [ "x-remaining-quota",  "x-consumed-quota" ]:
                    print(f"{key}: {value}")
            print()

            return json.loads(response.text)
        else:
            print(f"Request failed with status code {response.status_code}")
        
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
    


def getPage( url, page ):

    try:
        response = requests.get(url + "?page=" + str(page) + "&page_size=100", headers=headers)
    
        if response.status_code == 200:
            return json.loads( response.text ) 
        else:
            print(f"Request failed with status code {response.status_code}")
    
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")


def getAllPages( type ):
    projects = []

    project_url = api_url + endpoints[type]
    page = 1

    p = getPage( project_url, page )
    projects = p["entries"]

    while p["has_more"]:
        page += 1
        p = getPage( project_url, page )
        projects = projects + p["entries"]

    return projects

def getUsers():
    return getAllPages("users")

def getProjects():
    return getAllPages("projects")
    
def getTasks():
    return getAllPages("tasks")

def getCompanies():
    return getAllPages("companies")


# Speicherung in den Variablen um weniger API Calls zu produzieren

projects = getProjects()
tasks = getTasks()
companies = getCompanies()

def getProject(id):
    for p in projects:
        if p["id"] == id:
            return p

def getTask(id):
    for t in tasks:
        if t["id"] == id:
            return t

def getCompany(id):
    for c in companies:
        if c["id"] == id:
            return c

def getProjectsForCompany(company_id):
    cp = []
    for p in projects:
        if p["company_id"] == company_id:
            cp.append(p)
    return cp

def getTasksForProject(project_id):
    pt = []
    for t in tasks:
        if t["project_id"] == project_id:
            pt.append(t)
    return pt

def print_tree_of_tasks():
    for c in companies:
        print(c["name"], c["id"])
        for p in getProjectsForCompany( c["id"] ):
            print("\t",p["name"])
            for t in getTasksForProject( p["id"] ):
                print("\t\t",t["name"])        

# Check ob Kalendereinträge sich auf Papierkram zuordnen lässt

In diesem Notebook werden nur Time Entries angelegt, keine Companies, Projects oder Tasks. Das ist sicherer wenn man das per Hand in Papierkram macht.

In [ ]:

ready = []
missing = []

for entry in time_entries:
    if str(entry["company"]) in short_companies.keys():
        pc = getProjectsForCompany( short_companies[ str(entry["company"]) ]["id"] )
        found = False
        for p in pc:
            if entry["project"] == p["name"]:
                # check tasks
                tp = getTasksForProject( p["id"] )
                tfound = False
                for t in tp:
                    if entry["task"] == t["name"]:
                        ready.append( "✅ " + entry["company"] +  " - " +  entry["project"] +  " - " +  entry["task"] )
                        entry["task_id"] = t["id"]
                        # book here
                        tfound = True
                if not tfound:
                    missing.append( f'{entry["company"]} - {entry["project"]} - {entry["task"]}' )
                    #print( "Task", entry["task"], "for project", entry["project"], "for",  entry["company"], "missing" )
                found = True
        if not found:
            missing.append( f'{entry["company"]} - {entry["project"]}' )
    else:
        missing.append( f'{entry["company"]}' )


ready = sorted(ready)
if len(ready) > 0:
    print("Ready:")
    for r in ready:
        print(r)
        
# check if everything is fine
if len(missing) > 0:
    print("Missing:")
    missing = list(set(missing))
    for m in missing:
        print(m)
else:
    print("Ready to book")

# Time Entry Objekte erzeugen und buchen

So sehen die Time Entry Objekte aus:

```json
{
  "entry_date": "2020-05-11",
  "started_at_time": "10:00",
  "ended_at_time": "12:00",
  "comments": "Worked hard",
  "unbillable": false,
  "task": {
    "id": 97
  },
  "user": {
    "id": 1
  }
}
```

In [ ]:

# build (add the task id) and send all entries to papierkram

to_be_booked = []

for t in time_entries:

    task_object = {
        "id": t["task_id"]
    }
    user_object = {
        "id": USER_ID
    }
    
    api_time_entry_object = {
        
      "entry_date": t["entry_date"],
      "started_at_time": t["started_at_time"],
      "ended_at_time": t["ended_at_time"],
        
      "comments": t["comments"],
      #"billable_duration": t["duration"],
      #"unbillable": False,
      "task": task_object,
      "user": user_object,
    }

    to_be_booked.append( api_time_entry_object )
    
print("ready to book", len(to_be_booked), "entries")


# Buchen

In [ ]:
for t in to_be_booked:
    # print(json.dumps(t,indent=4))
    bookTimeEntry(t)